In [ ]:
from __future__ import print_function

import glob
import nibabel
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import skimage.transform
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import pickle
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [ ]:
raw_path = Path('.')
computed_path = Path('../models/clean')

In [ ]:
patients = sorted(glob.glob(str(raw_path / '*' / '*')))
print(f"{len(patients)} patients found.")

In [ ]:
with (computed_path / 'train.pickle').open('rb') as file:
    train = pickle.load(file)
with (computed_path / 'test.pickle').open('rb') as file:
    test = pickle.load(file)
X = train['X'] + test['X']
y = train['y'] + test['y']
if (computed_path / 'results.pickle').exists():
    with (computed_path / 'results.pickle').open('rb') as file:
        results = pickle.load(file)
    predicted = [None] * len(train['X']) + results['raw']
else:
    results = None
    predicted = None
generated_patients = train['patients'] + test['patients']

In [ ]:
def show_image(patient):
    patient = patients[patient]
    print(patient)
    t1 = nibabel.load(raw_path / patient / 'pre' / 'T1.nii.gz').get_fdata()
    flair = nibabel.load(raw_path / patient / 'pre' / 'FLAIR.nii.gz').get_fdata()
    gt = nibabel.load(raw_path / patient / 'wmh.nii.gz').get_fdata()
    
    patient_index = None
    if results:
        if patient in test['patients']:
            patient_index = test['patients'].index(patient)
    
    assert t1.shape == flair.shape and t1.shape == gt.shape
    def f(s):
        plt.subplot(1, 4, 1)
        plt.title('T1')
        plt.imshow(t1[:, :, s])
        plt.subplot(1, 4, 2)
        plt.title('FLAIR')
        plt.imshow(flair[:, :, s])
        plt.subplot(1, 4, 3)
        plt.title('Ground Truth')
        plt.imshow(gt[:, :, s], vmin=0, vmax=1)
        plt.colorbar()
        if patient_index is not None:
            plt.subplot(1, 4, 4)
            plt.title('Predicted')
            plt.imshow(results['transformed'][patient_index][s], vmin=0, vmax=1)
            plt.colorbar()
        plt.show()
    return interact(f, s=IntSlider(min=0, max=t1.shape[-1]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(patients)-1, value=0))

In [ ]:
def show_image(patient):
    print(generated_patients[patient])
    def f(s):
        plt.subplot(1, 5, 1)
        plt.title('T1')
        plt.imshow(X[patient][s, :, :, 0])
        plt.subplot(1, 5, 2)
        plt.title('FLAIR')
        plt.imshow(X[patient][s, :, :, 1])
        plt.subplot(1, 5, 3)
        plt.title('Preprocess')
        plt.imshow(X[patient][s, :, :, 2])
        plt.subplot(1, 5, 4)
        plt.title('Ground Truth')
        plt.imshow(y[patient][s, :, :], vmin=0, vmax=1)
        plt.colorbar()
        plt.subplot(1, 5, 5)
        plt.title('Predicted')
        if predicted and predicted[patient] is not None:
            plt.imshow(predicted[patient][s, :, :], vmin=0, vmax=1)
            plt.colorbar()
        plt.show()
    return interact(f, s=IntSlider(min=0, max=X[patient].shape[0]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(X)-1, value=0))

In [ ]:
predicted